In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr  05 20:20:02 2020
爬虫脚本：佛山链家房源信息
@author: yihao.he
"""
from bs4 import BeautifulSoup  
import pandas as pd
from tqdm import tqdm
import math
import requests  
import lxml
import re
import time




area_dic = {'禅城区':'chancheng',
            '南海区':'nanhai',
            '顺德区':'shunde',
            '三水区':'sanshui1',
            '高明区':'gaoming1'}


# 加个header以示尊敬
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.146 Safari/537.36',
           'Referer': 'https://fs.lianjia.com/ershoufang/'}

# 新建一个会话
sess = requests.session()
sess.get('https://fs.lianjia.com/ershoufang/', headers=headers)

# url示例：https://fs.lianjia.com/ershoufang/luohuqu/pg2/
url = 'https://fs.lianjia.com/ershoufang/{}/pg{}/'




# 当正则表达式匹配失败时，返回默认值（errif）
def re_match(re_pattern, string, errif=None):
    try:
        return re.findall(re_pattern, string)[0].strip()
    except IndexError:
        return errif



# 新建一个DataFrame存储信息
data = pd.DataFrame()

for key_, value_ in area_dic.items():
    # 获取该行政区下房源记录数
    start_url = 'https://fs.lianjia.com/ershoufang/{}/'.format(value_)
    html = sess.get(start_url).text
    house_num = re.findall('共找到<span> (.*?) </span>套.*二手房', html)[0].strip()
    print('💚{}: 二手房源共计「{}」套'.format(key_, house_num))
    time.sleep(1)
    # 页面限制🚫 每个行政区只能获取最多100页共计3000条房源信息
    total_page = int(math.ceil(min(3000, int(house_num)) / 30.0))
    for i in tqdm(range(total_page), desc=key_):
        html = sess.get(url.format(value_, i+1)).text
        soup = BeautifulSoup(html, 'lxml')
        info_collect = soup.find_all(class_="info clear")
        
        for info in info_collect:
            info_dic = {}
            # 行政区
            info_dic['area'] = key_
            # 房源的标题
            info_dic['title'] = re_match('target="_blank">(.*?)</a><!--', str(info))
            # 小区名
            info_dic['community'] = re_match('xiaoqu.*?target="_blank">(.*?)</a>', str(info))
            # 位置
            info_dic['position'] = re_match('<a href.*?target="_blank">(.*?)</a>.*?class="address">', str(info))
            # 税相关，如房本满5年
            info_dic['tax'] = re_match('class="taxfree">(.*?)</span>', str(info))
            # 总价
            info_dic['total_price'] = float(re_match('class="totalPrice"><span>(.*?)</span>万', str(info)))
            # 单价
            info_dic['unit_price'] = float(re_match('data-price="(.*?)"', str(info)))
            
            # 匹配房源标签信息，通过|切割
            # 包括面积，朝向，装修等信息
            icons = re.findall('class="houseIcon"></span>(.*?)</div>', str(info))[0].strip().split('|')
            info_dic['hourseType'] = icons[0].strip()
            info_dic['hourseSize'] = float(icons[1].replace('平米', ''))
            info_dic['direction'] = icons[2].strip()
            info_dic['fitment'] = icons[3].strip()
            
            # 存入DataFrame
            if data.empty:
                data = pd.DataFrame(info_dic,index=[0])
            else:
                data = data.append(info_dic,ignore_index=True)




# 去掉一条面积10000+平米的房源记录
data = data[data['hourseSize'] < 10000]
print(data.head())



💚禅城区: 二手房源共计「9067」套


禅城区: 100%|████████████████████████████████████████████████████████████████████████| 100/100 [02:53<00:00,  1.74s/it]


💚南海区: 二手房源共计「19898」套


南海区: 100%|████████████████████████████████████████████████████████████████████████| 100/100 [02:58<00:00,  1.78s/it]


💚顺德区: 二手房源共计「12625」套


...

💚三水区: 二手房源共计「4750」套


三水区: 100%|████████████████████████████████████████████████████████████████████████| 100/100 [03:42<00:00,  2.23s/it]


💚高明区: 二手房源共计「2004」套


高明区: 100%|██████████████████████████████████████████████████████████████████████████| 67/67 [02:36<00:00,  2.34s/it]

  area                       title community position    tax  total_price  \
0  禅城区     此房源处于电梯高层，精装修实用三房，通风采光好  雅居乐·曼克顿山       魁奇   None        215.0   
1  禅城区  雅居乐曼克顿山高层毛坯大四房 双地铁 含两个产权车位  雅居乐·曼克顿山       魁奇   None        500.0   
2  禅城区    祖庙街道普君新城步行街，普君市场，普君北路地铁。      普君新城       祖庙   None        115.0   
3  禅城区       时代云图 高层南向四房 方正实用带入户花园      时代云图       朝安  房本满五年        298.0   
4  禅城区   澜石地铁泷景，全新未住契税满两年，中高层望小区花园      泷景五期       澜石   None        180.0   

   unit_price hourseType  hourseSize direction fitment  
0     23964.0       3室2厅       89.72        西南      精装  
1     24631.0       5室2厅      203.00       南 北      毛坯  
2     18730.0       2室1厅       61.40       南 北      精装  
3     20819.0       4室2厅      143.14        西南      精装  
4     19517.0       3室2厅       92.23         北      精装  


In [2]:
#保存到 CSV
data.to_csv('.\\JN_DataWarehouse\\foshan_lianjiafangyuan\\lianjia-foshanfangyuan.csv')